In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, classification_report, make_scorer
from sklearn.ensemble import HistGradientBoostingClassifier

from imblearn.pipeline import Pipeline          # pip install imbalanced-learn
from imblearn.over_sampling import ADASYN

RANDOM_STATE = 42
DATA_DIR = Path(".")          # adjust if files live elsewhere
HR_FILE  = DATA_DIR / "heartrate_15min.csv"
ST_FILE  = DATA_DIR / "minuteStepsNarrow.csv"
DX_FILE  = DATA_DIR / "Diagnoses_20250404.csv"


In [2]:
# ---- diagnoses ---------------------------------------------
diag = (pd.read_csv(DX_FILE, parse_dates=["DCDate.diagnosis_baseline"])
          .rename(columns={"DCDate.diagnosis_baseline": "BaselineDate"})
          .dropna(subset=["BaselineDate"])
          [["PIDN", "BaselineDate", "Diagnosis_baseline_3groups"]])

# ---- heart-rate --------------------------------------------
hr = pd.read_csv(HR_FILE, parse_dates=["Time"])
common_pidn = set(diag.PIDN) & set(hr.PIDN)
diag = diag[diag.PIDN.isin(common_pidn)]
hr   = hr [hr .PIDN.isin(common_pidn)]

hr = hr.merge(diag[["PIDN", "BaselineDate"]], on="PIDN", how="left")
assert hr["BaselineDate"].notna().all()
print("HR rows:", len(hr), "| participants:", hr.PIDN.nunique())


HR rows: 506496 | participants: 192


In [3]:
# ---- helper: first N calendar days --------------------------
def first_n_days(g, bdate, n=14):
    after = g[g.Time.dt.date >= bdate]
    start = after.Time.dt.date.min() if not after.empty else g.Time.dt.date.min()
    end   = start + pd.Timedelta(days=n)
    return g[(g.Time.dt.date >= start) & (g.Time.dt.date < end)]

hr14 = pd.concat(
    [first_n_days(g, g["BaselineDate"].iloc[0].date(), n=14)
     for _, g in hr.groupby("PIDN")],
    ignore_index=True
)

# ---- daily mean HR per participant (for coupling later) ----
hr_daily = (hr14.assign(Date=hr14.Time.dt.date)
                   .groupby(["PIDN", "Date"])["Value"].mean()
                   .rename("HR_daily_mean")
                   .reset_index())

# ---- global 14-day HR stats --------------------------------
def hr_stats(df):
    v = df["Value"].to_numpy()
    n = v.size
    m = lambda a: np.nan if a.size == 0 else a.mean()
    return pd.Series({
        "hr_mean"   : m(v),
        "hr_std"    : np.std(v, ddof=0),
        "hr_min"    : v.min(),
        "hr_max"    : v.max(),
        "hr_iqr"    : np.percentile(v,75)-np.percentile(v,25),
        "hr_rmssd"  : np.sqrt(np.mean(np.diff(v)**2)) if n>1 else np.nan,
    })

features_hr = (hr14.groupby("PIDN")[["Time", "Value"]]
                   .apply(hr_stats)
                   .reset_index())
print("HR-feature table shape:", features_hr.shape)


HR-feature table shape: (192, 7)


In [4]:
# ---- read large steps CSV in chunks & keep study PIDNs -------
use_pids = set(features_hr.PIDN)
chunks = []
for chk in pd.read_csv(ST_FILE,
                       usecols=["PIDN","ActivityMinute","Steps"],
                       dtype={"PIDN":"int32","Steps":"int32"},
                       parse_dates=["ActivityMinute"],
                       chunksize=5_000_000):
    chunks.append(chk[chk.PIDN.isin(use_pids)])

steps_raw = pd.concat(chunks, ignore_index=True).rename(
    columns={"ActivityMinute":"Time","Steps":"Value"}
).merge(diag[["PIDN","BaselineDate"]], on="PIDN", how="left")

# ---- slice same 14-day window --------------------------------
step14 = pd.concat(
    [first_n_days(g, g["BaselineDate"].iloc[0].date(), n=14)
     for _, g in steps_raw.groupby("PIDN")],
    ignore_index=True
)

# ---- daily total steps --------------------------------------
step_daily = (step14.assign(Date=step14.Time.dt.date)
                       .groupby(["PIDN","Date"])["Value"].sum()
                       .rename("Steps_daily_total")
                       .reset_index())

# ---- step stats across 14 days ------------------------------
def step_stats(df):
    v = df["Steps_daily_total"].to_numpy()
    trend = v[-1]-v[0] if len(v)>=2 else np.nan
    return pd.Series({
        "steps_mean" : v.mean(),
        "steps_std"  : v.std(ddof=0),
        "steps_min"  : v.min(),
        "steps_max"  : v.max(),
        "steps_trend": trend
    })

features_st = (step_daily.groupby("PIDN")
                        .apply(step_stats)
                        .reset_index())

# ---- HR-to-steps coupling (Pearson r across 14 days) --------
coupling = (hr_daily.merge(step_daily, on=["PIDN","Date"], how="inner")
                   .groupby("PIDN")
                   .apply(lambda df: pd.Series({
                       "hr_steps_corr": np.corrcoef(df.HR_daily_mean,
                                                    df.Steps_daily_total)[0,1]
                                             if len(df)>=3 else np.nan}))
                   .reset_index())

print("Step-feature table shape:", features_st.shape)


Step-feature table shape: (190, 6)


C:\Users\14157\AppData\Local\Temp\ipykernel_36564\4276235692.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(step_stats)
C:\Users\14157\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\14157\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\14157\AppData\Local\Temp\ipykernel_36564\4276235692.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version 

In [5]:
# merge HR stats + step stats + coupling
features_full = (features_hr
                 .merge(features_st, on="PIDN", how="left")
                 .merge(coupling,   on="PIDN", how="left")
                 .merge(diag[["PIDN","Diagnosis_baseline_3groups"]], on="PIDN"))

features_full.to_csv("hr_steps_14day_features.csv", index=False)
print("Merged feature table saved | shape =", features_full.shape)


Merged feature table saved | shape = (192, 14)


In [7]:
# ================================================================
# 10-fold CV  +  wider HistGB grid  (ADASYN oversampling kept)
# ================================================================
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, make_scorer
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

# ---- train / test split (20 %) ---------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)

# ---- pipeline ---------------------------------------------------
pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale" , StandardScaler()),
    ("adasyn", ADASYN(random_state=RANDOM_STATE, sampling_strategy="auto")),
    ("model" , HistGradientBoostingClassifier(random_state=RANDOM_STATE))
])

# ---- wider grid -------------------------------------------------
param_grid = {
    "model__learning_rate"   : [0.01, 0.05, 0.1],
    "model__max_depth"       : [None, 3, 5],
    "model__l2_regularization": [0.0, 0.1, 0.5]
    # add 'model__max_leaf_nodes' or 'min_samples_leaf' if desired
}

cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

gs = GridSearchCV(
    pipe,
    param_grid,
    cv=cv10,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1,
    verbose=1
)

gs.fit(X_train, y_train)

print("\nBest 10-fold CV balanced-accuracy:", round(gs.best_score_, 3))
print("Best parameters:", gs.best_params_)

# ---- evaluate on held-out test set -----------------------------
y_pred = gs.predict(X_test)
print("\nTEST balanced-accuracy:", round(balanced_accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred))


Fitting 10 folds for each of 27 candidates, totalling 270 fits

Best 10-fold CV balanced-accuracy: 0.308
Best parameters: {'model__l2_regularization': 0.0, 'model__learning_rate': 0.1, 'model__max_depth': None}

TEST balanced-accuracy: 0.424
                   precision    recall  f1-score   support

Clinically Normal       0.70      0.76      0.73        25
     FTD syndrome       0.29      0.40      0.33         5
           MCI/AD       0.20      0.11      0.14         9

         accuracy                           0.56        39
        macro avg       0.40      0.42      0.40        39
     weighted avg       0.53      0.56      0.54        39



In [6]:
X = features_full.drop(columns=["Diagnosis_baseline_3groups","PIDN"])
y = features_full["Diagnosis_baseline_3groups"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)

pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale" , StandardScaler()),
    ("adasyn", ADASYN(random_state=RANDOM_STATE, sampling_strategy="auto")),
    ("model" , HistGradientBoostingClassifier(random_state=RANDOM_STATE))
])

param_grid = {
    "model__learning_rate": [0.05, 0.1],
    "model__max_depth"    : [None, 3]
}

gs = GridSearchCV(
    pipe, param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1, verbose=1
)
gs.fit(X_train, y_train)

print("\nBest CV balanced-accuracy:", round(gs.best_score_,3))
print("Best params:", gs.best_params_)

y_pred = gs.predict(X_test)
print("\nTEST balanced-accuracy:", round(balanced_accuracy_score(y_test, y_pred),3))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best CV balanced-accuracy: 0.303
Best params: {'model__learning_rate': 0.1, 'model__max_depth': None}

TEST balanced-accuracy: 0.424
                   precision    recall  f1-score   support

Clinically Normal       0.70      0.76      0.73        25
     FTD syndrome       0.29      0.40      0.33         5
           MCI/AD       0.20      0.11      0.14         9

         accuracy                           0.56        39
        macro avg       0.40      0.42      0.40        39
     weighted avg       0.53      0.56      0.54        39



In [8]:
# ================================================================
# Binary screen: Clinically Normal (0) vs Abnormal (1)
# Features: 14-day HR statistics + daily-step statistics + HR↔steps coupling
# ================================================================
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, make_scorer

RANDOM_STATE = 42

# ----- 1.  X / y (binary) -------------------------------------
X = features_full.drop(columns=["Diagnosis_baseline_3groups", "PIDN"])
y = (features_full["Diagnosis_baseline_3groups"] != "Clinically Normal").astype(int)

print("Class balance:", y.value_counts().to_dict())  # sanity-check

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)

# ----- 2.  pipeline (no ADASYN) -------------------------------
pipe_bin = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale" , StandardScaler()),
    ("model" , HistGradientBoostingClassifier(
                  random_state=RANDOM_STATE,
                  class_weight="balanced"))
])

param_grid = {
    "model__learning_rate"   : [0.01, 0.05, 0.1],
    "model__max_depth"       : [None, 3, 5],
    "model__l2_regularization": [0.0, 0.1, 0.5],
}

cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

gs = GridSearchCV(
    pipe_bin,
    param_grid,
    cv=cv10,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1,
    verbose=1
)
gs.fit(X_train, y_train)

print("\nBest 10-fold CV balanced-accuracy:", round(gs.best_score_, 3))
print("Best parameters:", gs.best_params_)

# ----- 3.  evaluate on held-out test set -----------------------
y_pred = gs.predict(X_test)
print("\nTEST balanced-accuracy:", round(balanced_accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred, target_names=["CN", "Abnormal"]))


Class balance: {0: 122, 1: 70}
Fitting 10 folds for each of 27 candidates, totalling 270 fits

Best 10-fold CV balanced-accuracy: 0.588
Best parameters: {'model__l2_regularization': 0.0, 'model__learning_rate': 0.01, 'model__max_depth': None}

TEST balanced-accuracy: 0.514
              precision    recall  f1-score   support

          CN       0.65      0.60      0.62        25
    Abnormal       0.38      0.43      0.40        14

    accuracy                           0.54        39
   macro avg       0.51      0.51      0.51        39
weighted avg       0.55      0.54      0.54        39



In [9]:
# ================================================================
# Binary screen: Clinically Normal (0) vs Abnormal (1)
# Features: 14-day HR statistics + daily-step statistics + HR↔steps coupling
# ================================================================
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, make_scorer

RANDOM_STATE = 42

# ----- 1.  X / y (binary) -------------------------------------
X = features_full.drop(columns=["Diagnosis_baseline_3groups", "PIDN"])
y = (features_full["Diagnosis_baseline_3groups"] != "Clinically Normal").astype(int)

print("Class balance:", y.value_counts().to_dict())  # sanity-check

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)

# ----- 2.  pipeline (no ADASYN) -------------------------------
pipe_bin = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale" , StandardScaler()),
    ("model" , HistGradientBoostingClassifier(
                  random_state=RANDOM_STATE,
                  class_weight="balanced"))
])

param_grid = {
    "model__learning_rate"   : [0.01, 0.05],
    "model__max_depth"       : [None, 5],
    "model__min_samples_leaf": [10, 20, 30],
    "model__class_weight"    : [{0:1, 1:2}, {0:1, 1:3}]
}


cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

gs = GridSearchCV(
    pipe_bin,
    param_grid,
    cv=cv10,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1,
    verbose=1
)
gs.fit(X_train, y_train)

print("\nBest 10-fold CV balanced-accuracy:", round(gs.best_score_, 3))
print("Best parameters:", gs.best_params_)

# ----- 3.  evaluate on held-out test set -----------------------
y_pred = gs.predict(X_test)
print("\nTEST balanced-accuracy:", round(balanced_accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred, target_names=["CN", "Abnormal"]))

Class balance: {0: 122, 1: 70}
Fitting 10 folds for each of 24 candidates, totalling 240 fits

Best 10-fold CV balanced-accuracy: 0.63
Best parameters: {'model__class_weight': {0: 1, 1: 2}, 'model__learning_rate': 0.01, 'model__max_depth': None, 'model__min_samples_leaf': 20}

TEST balanced-accuracy: 0.546
              precision    recall  f1-score   support

          CN       0.68      0.52      0.59        25
    Abnormal       0.40      0.57      0.47        14

    accuracy                           0.54        39
   macro avg       0.54      0.55      0.53        39
weighted avg       0.58      0.54      0.55        39



In [11]:
# ================================================================
# Threshold-tuning utility  (binary CN vs Abnormal)
# ================================================================
from sklearn.metrics import (
    roc_curve, balanced_accuracy_score, confusion_matrix, recall_score
)
import pandas as pd

# 1. get class-1 probabilities on the held-out test set
probs = gs.best_estimator_.predict_proba(X_test)[:, 1]

# 2. ROC curve  → tpr (recall for class 1) / fpr / thresholds
fpr, tpr, thr = roc_curve(y_test, probs)

# 3. balanced-accuracy for every threshold
bal_acc = []
for t in thr:
    y_pred = (probs >= t).astype(int)
    bal_acc.append(balanced_accuracy_score(y_test, y_pred))
bal_acc = np.array(bal_acc)

# 4-a. threshold that maximises balanced-accuracy
idx_best = bal_acc.argmax()
best_cut = thr[idx_best]

# 4-b. lowest threshold that gives ≥65 % recall for Abnormal
target_recall = 0.65
idx_recall = np.where(tpr >= target_recall)[0]
cut_recall = thr[idx_recall[0]] if len(idx_recall) else best_cut   # fallback

# 5. evaluate both thresholds
def eval_cut(cut, label):
    y_pred = (probs >= cut).astype(int)
    print(f"\n--- {label} threshold = {cut:.3f} ---")
    print("Balanced-acc :", round(balanced_accuracy_score(y_test, y_pred), 3))
    print("Abnormal recall :", round(recall_score(y_test, y_pred), 3))
    print("Confusion matrix [CN  Abn]:\n", confusion_matrix(y_test, y_pred))

eval_cut(best_cut, "Max-BA")
eval_cut(cut_recall, "≥65 % Recall")



--- Max-BA threshold = 0.571 ---
Balanced-acc : 0.579
Abnormal recall : 0.357
Confusion matrix [CN  Abn]:
 [[20  5]
 [ 9  5]]

--- ≥65 % Recall threshold = 0.466 ---
Balanced-acc : 0.577
Abnormal recall : 0.714
Confusion matrix [CN  Abn]:
 [[11 14]
 [ 4 10]]


In [12]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, recall_score, confusion_matrix
import pandas as pd

# predicted probs from current model
probs = gs.best_estimator_.predict_proba(X_test)[:, 1]

results = []
for cut in np.arange(0.40, 0.61, 0.01):
    y_pred = (probs >= cut).astype(int)
    ba  = balanced_accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    fp  = ((y_pred == 1) & (y_test == 0)).sum()
    fn  = ((y_pred == 0) & (y_test == 1)).sum()
    results.append([round(cut, 2), round(ba, 3), round(rec, 3), fp, fn])

df_thr = pd.DataFrame(results,
                      columns=["cut", "bal_acc", "abn_recall", "false_pos", "false_neg"])
display(df_thr)

best_row  = df_thr.loc[df_thr["bal_acc"].idxmax()]
rec65_row = df_thr[df_thr["abn_recall"] >= 0.65].head(1)

print("\nBest BA :", best_row.to_dict())
if not rec65_row.empty:
    print("≥65 % recall :", rec65_row.iloc[0].to_dict())
else:
    print("No threshold hit 0.65 recall in scanned range.")


,cut,bal_acc,abn_recall,false_pos,false_neg
0,0.40,0.417,0.714,22,4
1,0.41,0.417,0.714,22,4
2,0.42,0.437,0.714,21,4
3,0.43,0.497,0.714,18,4
4,0.44,0.497,0.714,18,4
5,0.45,0.537,0.714,16,4
6,0.46,0.577,0.714,14,4
7,0.47,0.541,0.643,14,5
8,0.48,0.546,0.571,12,6
9,0.49,0.546,0.571,12,6



Best BA : {'cut': 0.57, 'bal_acc': 0.579, 'abn_recall': 0.357, 'false_pos': 5.0, 'false_neg': 9.0}
≥65 % recall : {'cut': 0.4, 'bal_acc': 0.417, 'abn_recall': 0.714, 'false_pos': 22.0, 'false_neg': 4.0}


In [14]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import balanced_accuracy_score, recall_score, confusion_matrix
import numpy as np, pandas as pd

# 1️⃣  wrap the trained pipeline in an isotonic calibrator (5-fold CV on training data)
cal_model = CalibratedClassifierCV(
    estimator=gs.best_estimator_,     # ← changed keyword
    method="isotonic",
    cv=5
)
cal_model.fit(X_train, y_train)

# 2️⃣  calibrated probabilities on the test set
probs_cal = cal_model.predict_proba(X_test)[:, 1]

# 3️⃣  scan thresholds 0.40-0.60 by 0.01
records = []
for cut in np.arange(0.40, 0.61, 0.01):
    y_pred = (probs_cal >= cut).astype(int)
    ba  = balanced_accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    records.append([round(cut,2), round(ba,3), round(rec,3)])

df_cal = pd.DataFrame(records, columns=["cut","bal_acc","abn_recall"])
display(df_cal)

best_row  = df_cal.loc[df_cal["bal_acc"].idxmax()]
rec65_row = df_cal[df_cal["abn_recall"] >= 0.65].head(1)

print("\n(ISO) Best BA :", best_row.to_dict())
print("(ISO) ≥65 % recall :", rec65_row.iloc[0].to_dict() if not rec65_row.empty else "none")


,cut,bal_acc,abn_recall
0,0.40,0.474,0.429
1,0.41,0.439,0.357
2,0.42,0.423,0.286
3,0.43,0.463,0.286
4,0.44,0.447,0.214
5,0.45,0.467,0.214
6,0.46,0.467,0.214
7,0.47,0.547,0.214
8,0.48,0.547,0.214
9,0.49,0.547,0.214



(ISO) Best BA : {'cut': 0.53, 'bal_acc': 0.551, 'abn_recall': 0.143}
(ISO) ≥65 % recall : none
